In [1]:
from google.colab import files
uploaded = files.upload()
import zipfile

# Unzip the file
with zipfile.ZipFile("credits.csv.zip", 'r') as zip_ref:
    zip_ref.extractall()
import pandas as pd

# Load the credits data
credits = pd.read_csv("credits.csv")

# Preview
credits.head()


Saving credits.csv.zip to credits.csv (1).zip


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [2]:
def get_top_cast(obj):
    try:
        L = []
        for i in ast.literal_eval(obj):
            L.append(i['name'])
        return L[:3]  # Top 3 cast members
    except:
        return []

def get_director(obj):
    try:
        for i in ast.literal_eval(obj):
            if i['job'] == 'Director':
                return i['name']
    except:
        return ''


credits['top_cast'] = credits['cast'].apply(get_top_cast)
credits['director'] = credits['crew'].apply(get_director)



In [3]:
def clean_names(x):
    if isinstance(x, list):
        return [i.replace(" ", "").lower() for i in x]
    if isinstance(x, str):
        return x.replace(" ", "").lower()
    return ''

credits['top_cast'] = credits['top_cast'].apply(clean_names)
credits['director'] = credits['director'].apply(clean_names)

credits['soup'] = credits['top_cast'].apply(lambda x: ' '.join(x)) + ' ' + credits['director']


In [5]:
# Check if any soup rows are empty or null
print(credits['soup'].isnull().sum())
print(credits['soup'].apply(lambda x: len(x.strip()) == 0).sum())


0
45476


In [8]:
import pandas as pd
import ast

credits = pd.read_csv("/content/credits.csv")

def get_top_cast(obj):
    try:
        return [i['name'] for i in ast.literal_eval(obj)][:3]
    except:
        return []

def get_director(obj):
    try:
        for i in ast.literal_eval(obj):
            if i['job'] == 'Director':
                return i['name']
    except:
        return ''

credits['top_cast'] = credits['cast'].apply(get_top_cast)
credits['director'] = credits['crew'].apply(get_director)

def clean_text(x):
    if isinstance(x, list):
        return [i.replace(" ", "").lower() for i in x]
    elif isinstance(x, str):
        return x.replace(" ", "").lower()
    else:
        return ''

credits['top_cast'] = credits['top_cast'].apply(clean_text)
credits['director'] = credits['director'].apply(clean_text)


In [9]:
credits['soup'] = credits['top_cast'].apply(lambda x: ' '.join(x)) + ' ' + credits['director']
credits['soup'] = credits['soup'].fillna('')

# Optional: check one soup
print(credits['soup'].iloc[0])


tomhanks timallen donrickles johnlasseter


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(credits['soup'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)


In [18]:
with zipfile.ZipFile("movies_metadata.csv.zip", 'r') as zip_ref:
    zip_ref.extractall()
import pandas as pd

# Load the credits data
credits = pd.read_csv("movies_metadata.csv")

# Load both datasets
credits = pd.read_csv("/content/credits.csv")
movies = pd.read_csv("/content/movies_metadata.csv", low_memory=False)

# Convert ID column to numeric (handle errors)
movies['id'] = pd.to_numeric(movies['id'], errors='coerce')

# Merge on 'id'
credits['id'] = pd.to_numeric(credits['id'], errors='coerce')
merged = credits.merge(movies[['id', 'title']], on='id')


<ipython-input-18-2562288835>:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  credits = pd.read_csv("movies_metadata.csv")


In [19]:
# Replace everywhere
credits = merged


In [20]:
# Apply functions to clean cast and director
credits['top_cast'] = credits['cast'].apply(get_top_cast)
credits['director'] = credits['crew'].apply(get_director)

credits['top_cast'] = credits['top_cast'].apply(clean_text)
credits['director'] = credits['director'].apply(clean_text)

# Build the soup
credits['soup'] = credits['top_cast'].apply(lambda x: ' '.join(x)) + ' ' + credits['director']
credits['soup'] = credits['soup'].fillna('')

# Vectorize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(credits['soup'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# Reset index and build reverse map
credits = credits.reset_index(drop=True)
indices = pd.Series(credits.index, index=credits['title'].str.lower())

# Recommend function
def recommend(title, cosine_sim=cosine_sim):
    title = title.lower()
    if title not in indices:
        return f"'{title}' not found in the dataset."

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]

    return credits[['title', 'top_cast', 'director']].iloc[movie_indices]


In [22]:
indices['the dark knight']

credits = credits.drop_duplicates(subset='title')
credits = credits.reset_index(drop=True)

# Rebuild the indices after removing duplicates
indices = pd.Series(credits.index, index=credits['title'].str.lower())

recommend("The Dark Knight")


,title,top_cast,director
6989,An Amazing Couple,"[ornellamuti, valériemairesse, catherinefrot]",lucasbelvaux
8685,Street Smart,"[christopherreeve, kathybaker, mimirogers]",jerryschatzberg
17478,The Cost of Living,"[vincentlindon, fabriceluchini, géraldinepailhas]",philippeleguay
39781,Saint Seiya: Legend of Crimson Youth,"[tōrufuruya, hirotakasuzuoki, hideyukihori]",shigeyasuyamauchi
60,Eye for an Eye,"[sallyfield, kiefersutherland, edharris]",johnschlesinger
